In [3]:
import os
import rasterio
import numpy as np

def reclassify_raster(input_raster_path, output_raster_path, reclass_rules, output_nodata_val=-9999):
    """
    Reclassifies a raster based on a list of rules.

    Args:
        input_raster_path (str): The full path to the input raster file.
        output_raster_path (str): The full path to save the output reclassified raster file.
        reclass_rules (list): A list of lists, where each inner list is [from_value, to_value, new_value].
                              Values in the input raster falling within (from_value, to_value] will be
                              assigned the new_value.
        output_nodata_val (int): The NoData value to use for the output raster.
    """
    if not os.path.exists(input_raster_path):
        print(f"!!! Error: Input raster file not found at '{input_raster_path}'.")
        return

    print(f"\n--- Starting Reclassification for: {os.path.basename(input_raster_path)} ---")
    
    try:
        with rasterio.open(input_raster_path) as src:
            data = src.read(1)
            meta = src.meta.copy()
            input_nodata_val = src.nodata
            
            # Prepare conditions and choices for np.select
            conditions = []
            choices = []
            print("Applying reclassification rules:")
            for rule in reclass_rules:
                from_val, to_val, new_val = rule
                print(f"  - Values from {from_val} to {to_val} will be reclassified to {new_val}")
                conditions.append((data > from_val) & (data <= to_val))
                choices.append(new_val)
            
            # Apply reclassification using np.select
            reclassified_data = np.select(conditions, choices, default=output_nodata_val)
            
            # Handle NoData values from the input raster
            if input_nodata_val is not None:
                reclassified_data[data == input_nodata_val] = output_nodata_val

            # Update metadata for the output raster
            meta.update(
                dtype=rasterio.int16, # Suitability scores are integers
                count=1,
                nodata=output_nodata_val,
                compress='lzw' # Add LZW compression
            )
            
            # Write the reclassified data to a new GeoTIFF file
            with rasterio.open(output_raster_path, 'w', **meta) as dst:
                dst.write(reclassified_data.astype(rasterio.int16), 1)
                
            print(f"Successfully reclassified raster and saved to: {output_raster_path}")

    except Exception as e:
        print(f"An error occurred during reclassification: {e}")

# --- MAIN SCRIPT EXECUTION FOR DEPRESSION RECLASSIFICATION ---
if __name__ == "__main__":
    # --- USER: VERIFY YOUR INPUTS AND RULES HERE ---

    # 1. Path to your input Depression Depth raster
    # This should be the output from your QGIS Raster Calculator step (DEM_Filled - DEM_Merged_Final)
    input_raster_path = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/Suitability_Layers/Depression_Depth_Corrected.tif"

    # 2. Path to save the new reclassified Depression Suitability raster
    output_raster_path = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/Suitability_Layers/Depression_Suitability.tif"
    
    # Create the output directory if it doesn't exist
    output_dir = os.path.dirname(output_raster_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 3. Define the reclassification rules for Depression Depth.
    # Format: [from_value, to_value, new_suitability_score]
    # Deeper depressions should get higher suitability scores.
    # ADJUST THESE VALUES BASED ON YOUR Depression_Depth_Corrected.tif HISTOGRAM!
    # (Example values, based on your previous histogram where max was ~24)
    depression_reclass_rules = [
        [0, 1, 1],      # Very shallow or no depressions (Least Suitable)
        [1, 5, 2],
        [5, 10, 3],
        [10, 20, 4],
        [20, 99999, 5]  # Deepest depressions (Most Suitable) - use a very large number for upper bound
    ]

    # --- END OF USER SETTINGS ---

    # --- Run the Reclassification ---
    reclassify_raster(input_raster_path, output_raster_path, depression_reclass_rules)
    
    print("\nDepression reclassification process finished.")


--- Starting Reclassification for: Depression_Depth_Corrected.tif ---
Applying reclassification rules:
  - Values from 0 to 1 will be reclassified to 1
  - Values from 1 to 5 will be reclassified to 2
  - Values from 5 to 10 will be reclassified to 3
  - Values from 10 to 20 will be reclassified to 4
  - Values from 20 to 99999 will be reclassified to 5
Successfully reclassified raster and saved to: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Processed_Rasters/Suitability_Layers/Depression_Suitability.tif

Depression reclassification process finished.
